In [1]:
from utils import DataLoader

eeg = DataLoader(1,(1,9))

done with p


In [2]:
eegtuple, emgtuple = eeg.load_data_for_nn(participant=1, sigtype='both')

In [3]:
emgtuple[0].shape, emgtuple[1].shape, emgtuple[2].shape, eegtuple[0].shape, eegtuple[1].shape, eegtuple[2].shape

((32920, 11, 294), (294, 3), (294, 3), (4115, 38, 294), (294, 3), (294, 3))